In [1]:
# %pip install numpy 
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# %pip install pandas  
# %pip install huggingFace
# %pip install transformers
# %pip install nltk
# %pip install scikit-learn
# %pip install datasets
# %pip install pickle

In [ ]:
import re
# Numpy 
import numpy as np
# Pickle
import pickle
# Pandas
import pandas as pd
# Hugging Face
import huggingface_hub
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
# PyTorch
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# SkLearn
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# nltk.download()
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Steve\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Steve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
# Load datasets 
# Hate Xplain
hate_xplain = pd.read_csv("data/hate_xplain.csv")

# Implicit Hate 
implicit_hate = pd.read_csv('data/implicit-hate-corpus/implicit_hate_v1_stg2_posts.tsv', delimiter='\t')
label_map = {
    'white_grievance': 0, 'incitement': 1, 'inferiority': 2,
    'irony': 3, 'stereotypical': 4, 'threatening': 5, 'other': 6
}

implicit_hate['class_label'] = implicit_hate['implicit_class'].map(label_map)
implicit_hate.drop("extra_implicit_class", axis=1, inplace=True)

# Toxic-Spans
annotations = pd.read_csv('data/toxic-spans/annotations.csv')
comments = pd.read_csv('data/toxic-spans/comments.csv')

toxic_spans = pd.merge(annotations, comments, on='comment_id')

In [3]:
hate_xplain

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [4]:
implicit_hate

,post,implicit_class,class_label
0,""" : jewish harvard professor noel ignatiev w...",white_grievance,0
1,rt : tune in today's jmt for my interview w /...,irony,3
2,: : hate posters ? you want hate buddy ? read ...,white_grievance,0
3,: hunt for white women : black racist serial k...,stereotypical,4
4,: looks like he was a fan of the islamic jew-h...,stereotypical,4
...,...,...,...
6341,"Seeing prostitutes is morally wrong, but being...",irony,3
6342,I wonder how many females I raped today I spen...,irony,3
6343,Having a criminal record is more attractive to...,stereotypical,4
6344,Another advantage of being a female: getting b...,inferiority,2


In [5]:
toxic_spans

,annotation,comment_id,worker,country,all toxic,not toxic,comment_text
0,0,5167187,868,USA,False,False,That's right. They are not normal. And I am st...
1,1,5167187,1316,USA,False,False,That's right. They are not normal. And I am st...
2,2,5167187,1295,USA,False,True,That's right. They are not normal. And I am st...
3,3,5167187,2856,USA,False,False,That's right. They are not normal. And I am st...
4,4,5521110,418,VEN,True,True,"yep, this crap sounds like its from a libertarian"
...,...,...,...,...,...,...,...
59210,59759,6209917,2545,PER,True,True,Quite the contrary...because I carry every day...
59211,59760,6209917,2854,VEN,True,True,Quite the contrary...because I carry every day...
59212,59761,6209917,1531,VEN,True,True,Quite the contrary...because I carry every day...
59213,59762,6209917,2084,USA,False,False,Quite the contrary...because I carry every day...


In [6]:
def bow_preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    porter = PorterStemmer()

    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [porter.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

def bow_preprocess_data(data, label, vectorizer): 
    data = data.apply(bow_preprocess_text)
    X = vectorizer.fit_transform(data)
    y = label

    return X, y

In [ ]:
vectorizer = CountVectorizer()
# Hate Xplain
hx_X, hx_y = bow_preprocess_data(hate_xplain['tweet'], hate_xplain['class'], vectorizer)
hx_train_X, hx_test_X, hx_train_y, hx_test_y = train_test_split(hx_X, hx_y, test_size=0.2, random_state=42)
hx_n_classes = len(np.unique(hate_xplain['class']))

# Implicit Hate
ih_X, ih_y = bow_preprocess_data(implicit_hate['post'], implicit_hate['class_label'], vectorizer)
ih_train_X, ih_test_X, ih_train_y, ih_test_y = train_test_split(ih_X, ih_y, test_size=0.2, random_state=42)
ix_n_classes = len(np.unique(implicit_hate['class_label']))

# Toxic-Spans
ts_X, ts_y = bow_preprocess_data(toxic_spans['comment_text'], toxic_spans['all toxic'], vectorizer)
ts_train_X, ts_test_X, ts_train_y, ts_test_y = train_test_split(ts_X, ts_y, test_size=0.2, random_state=42)
ts_n_classes = len(np.unique(toxic_spans['all toxic']))


In [8]:
def evaluate(y_test, y_pred):
    classification = classification_report(y_test, y_pred, zero_division=1)
    return classification

In [9]:
def run(train_X, train_y, test_X, test_y, savepath: str): 
    model = MultinomialNB()
    model.fit(train_X, train_y)
    pickle.dump(model, open(savepath, 'wb'))
    y_pred = model.predict(test_X)
    stats = evaluate(test_y, y_pred)
    return stats 

def print_stats(stats, name: str):
    print(f"Stats for {name}")
    print(stats)



In [10]:
hx_stats = run(hx_train_X, hx_train_y, hx_test_X, hx_test_y, savepath='NB/hate_explain_NB.pkl')
print_stats(hx_stats, "Hate Xplain")

Stats for Hate Xplain
              precision    recall  f1-score   support

           0       0.34      0.09      0.14       290
           1       0.88      0.97      0.92      3832
           2       0.83      0.65      0.73       835

    accuracy                           0.86      4957
   macro avg       0.68      0.57      0.60      4957
weighted avg       0.84      0.86      0.84      4957



In [11]:
ih_stats = run(ih_train_X, ih_train_y, ih_test_X, ih_test_y, savepath='NB/implicit_hate_NB.pkl')
print_stats(ih_stats, "Implicit Hate")

Stats for Implicit Hate
              precision    recall  f1-score   support

           0       0.51      0.71      0.59       325
           1       0.54      0.49      0.51       253
           2       0.63      0.37      0.47       167
           3       0.57      0.40      0.47       169
           4       0.49      0.47      0.48       219
           5       0.49      0.62      0.55       120
           6       1.00      0.00      0.00        17

    accuracy                           0.52      1270
   macro avg       0.60      0.44      0.44      1270
weighted avg       0.54      0.52      0.51      1270



In [12]:
ts_stats = run(ts_train_X, ts_train_y, ts_test_X, ts_test_y, savepath='NB/toxic_spans_NB.pkl')
print_stats(ts_stats, "Toxic-Spans")

Stats for Toxic-Spans
              precision    recall  f1-score   support

       False       0.66      0.80      0.73      7491
        True       0.47      0.30      0.36      4352

    accuracy                           0.62     11843
   macro avg       0.56      0.55      0.54     11843
weighted avg       0.59      0.62      0.59     11843



In [23]:
bert = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(bert)
bert_model = DistilBertForSequenceClassification.from_pretrained(bert)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def encode_data(texts, labels, tokenizer, max_length=128):
    texts = [str(text) for text in texts]
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    dataset = torch.utils.data.TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels))
    return dataset

In [30]:
hx_dataset = encode_data(hate_xplain['tweet'], hate_xplain['class'], tokenizer)
hx_train, hx_test = train_test_split(hx_dataset, test_size=0.2, random_state=42)

In [32]:
def train(model, train_loader, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        for batch in train_loader: 
            input_ids, attn_mask, labels = [b for b in batch]
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attn_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

def evaluate(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader: 
            input_ids, attn_mask, labels = [b for b in batch]

            outputs = model(input_ids, attention_mask=attn_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    report = classification_report(all_labels, all_preds, target_names=["Hate Speech", "Normal", "Offensive"], output_dict=True)
    overall_acc = np.mean(np.array(all_preds) == np.array(all_labels))

    return report, overall_acc
            

In [33]:
optimizer = optim.AdamW(bert_model.parameters(), lr=5e-5)
train(bert_model, hx_train, optimizer, 3)

RuntimeError: The size of tensor a (128) must match the size of tensor b (512) at non-singleton dimension 1